# Exécuter un script d’entraînement en tant que travail de commande

Vous pouvez utiliser le SDK Python pour Azure Machine Learning afin de soumettre des scripts en tant que travaux de commande. En utilisant des travaux, vous pouvez facilement suivre les paramètres d’entrée et les sorties lors de l’entraînement d’un modèle Machine Learning.

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Suivi personnalisé avec MLflow

Lorsque vous exécutez un script en tant que travail, vous pouvez utiliser MLflow dans votre script d’entraînement pour suivre le modèle. MLflow vous permet de suivre tous les paramètres, métriques ou artefacts personnalisés que vous souhaitez stocker avec la sortie de votre travail.

Exécutez les cellules suivantes pour créer le script **train-model-mlflow.py** dans le dossier **src**. Le script entraîne un modèle de classification avec le fichier **diabetes.csv** dans le même dossier, qui est passé comme argument. 

Passez en revue le code ci-dessous pour vérifier que le script importera `mlflow` et journalisera :

- le taux de régularisation en tant que **paramètre** ; 
- la justesse et l’AUC en tant que **métriques** ;
- la courbe ROC tracée en tant qu’**artefact**.

In [ ]:
À présent, vous pouvez soumettre le script en tant que travail de commande.

Exécutez la cellule ci-dessous pour entraîner le modèle. 

In [ ]:
Dans le studio, accédez au travail **diabetes-train-mlflow** pour explorer la vue d’ensemble du travail de commande que vous avez exécuté :

- Les paramètres journalisés sont affichés sous l’onglet **Vue d’ensemble**, sous **Paramètres**.
- Les métriques journalisées sont affichées sous l’onglet **Métriques**.
- Les artefacts journalisés sont affichés sous l’onglet **Images** (spécifiquement pour les images) et sous l’onglet **Sorties + journaux** (pour tous les fichiers).

## Journalisation automatique avec MLflow

Au lieu d’utiliser la journalisation personnalisée, MLflow peut également enregistrer automatiquement tous les paramètres, métriques et artefacts. La journalisation automatique avec MLflow ne nécessite qu’une seule ligne de code.

Exécutez la cellule suivante pour créer le script **train-model-autolog.py** dans le dossier **src**. Le script entraîne un modèle de classification avec le fichier **diabetes.csv** dans le même dossier, qui est passé comme argument. 

Si vous examinez le code ci-dessous, vous voyez que le script importe `mlflow` et qu’il active la journalisation automatique avec la ligne : 

`mlflow.autolog()`

In [ ]:
À présent, vous pouvez soumettre le script en tant que travail de commande.

Exécutez la cellule ci-dessous pour entraîner le modèle. 

In [ ]:
Dans le studio, accédez au travail **diabetes-train-autolog** pour explorer la vue d’ensemble du travail de commande que vous avez exécuté :

- Les paramètres journalisés sont affichés sous l’onglet **Vue d’ensemble**, sous **Paramètres**.
- Les métriques journalisées sont affichées sous l’onglet **Métriques**.
- Les artefacts journalisés sont affichés sous l’onglet **Images** (spécifiquement pour les images) et sous l’onglet **Sorties + journaux** (pour tous les fichiers, y compris les fichiers de modèle).

## Utiliser MLflow pour afficher et rechercher des expériences

Azure Machine Learning studio est une interface utilisateur avec laquelle vous pouvez facilement afficher et comparer des exécutions de travaux. Vous pouvez également utiliser MLflow pour afficher des travaux d’expérience. 

Pour voir les travaux dans l’espace de travail, utilisez la commande suivante qui liste les expériences dans l’espace de travail :


In [ ]:
Vous pouvez récupérer une expérience spécifique par son nom :

Avec un nom d’expérience, vous pouvez récupérer tous les travaux de cette expérience :

Pour comparer plus facilement les exécutions de travaux et les sorties, vous pouvez configurer la recherche afin que les résultats soient classés. Par exemple, la cellule suivante classe les résultats selon la valeur `start_time` et affiche un maximum de `2` résultats : 

In [ ]:
Vous pouvez même créer une requête pour filtrer les exécutions. Les chaînes de requête de filtre sont écrites avec une version simplifiée de la clause SQL `WHERE`. 

Pour filtrer les exécutions, vous pouvez utiliser ces deux classes de comparateurs :

- Comparateurs numériques (métriques) : =, !=, >, >=, < et <=.
- Comparateurs de chaîne (params, étiquettes et attributs) : = et !=.

Découvrez-en plus sur le [suivi des expériences avec MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

Now, you can submit the script as a command job.

Run the cell below to train the model. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In the Studio, navigate to the **diabetes-train-autolog** job to explore the overview of the command job you ran:

- Find the logged parameters in the **Overview** tab, under **Params**.
- Find the logged metrics in the **Metrics** tab.
- Find the logged artifacts in the **Images** tab (specifically for images), and in the **Outputs + logs** tab (all files, including the model files).

## Use MLflow to view and search for experiments

The Azure Machine Learning Studio is an easy-to-use UI to view and compare job runs. Alternatively, you can use MLflow to view experiment jobs. 

To list the jobs in the workspace, use the following command to list the experiments in the workspace:


In [ ]:
import mlflow
experiments = mlflow.list_experiments()
for exp in experiments:
    print(exp.name)

To retrieve a specific experiment, you can get it by its name:

In [ ]:
experiment_name = "diabetes-training"
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

Using an experiment name, you can retrieve all jobs of that experiment:

In [ ]:
mlflow.search_runs(exp.experiment_id)

To more easily compare job runs and outputs, you can configure the search to order the results. For example, the following cell orders the results by `start_time`, and only shows a maximum of `2` results: 

In [ ]:
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)

You can even create a query to filter the runs. Filter query strings are written with a simplified version of the SQL `WHERE` clause. 

To filter, you can use two classes of comparators:

- Numeric comparators (metrics): =, !=, >, >=, <, and <=.
- String comparators (params, tags, and attributes): = and !=.

Learn more about [how to track experiments with MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

In [ ]:
query = "metrics.AUC > 0.8 and tags.model_type = 'LogisticRegression'"
mlflow.search_runs(exp.experiment_id, filter_string=query)